## Embedding model 의 문장 유사도 테스트
### [KoSimCSE-roberta](https://huggingface.co/BM-K/KoSimCSE-roberta)

In [32]:
import torch
from transformers import AutoModel, AutoTokenizer

def cal_score(a, b):
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100

model = AutoModel.from_pretrained('BM-K/KoSimCSE-roberta')
tokenizer = AutoTokenizer.from_pretrained('BM-K/KoSimCSE-roberta')

In [42]:
def embedding_similarity(model, tokenizer, sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    embeddings, _ = model(**inputs, return_dict=False)

    score01 = cal_score(embeddings[0][0], embeddings[1][0])
    score02 = cal_score(embeddings[0][0], embeddings[2][0])
    score03 = cal_score(embeddings[1][0], embeddings[2][0])
    
    score = [score01, score02, score03 ]
    max_score = max(score).item()
    max_score_idx = score.index(max(score))
    
    if max_score_idx == 0:
        print(f"1,2번 문장이 {max_score}로 가장 유사합니다.")
    elif max_score_idx == 1:  
        print(f"1,3번 문장이 {max_score}로 가장 유사합니다.")
    else:    
        print(f"2,3번 문장이 {max_score}로 가장 유사합니다.") 

    print('(', score01.item(), score02.item(), score03.item(), ')')

    return 

In [43]:
sentences = ['치타가 들판을 가로 질러 먹이를 쫓는다.',
             '치타 한 마리가 먹이 뒤에서 달리고 있다.',
             '원숭이 한 마리가 드럼을 연주한다.']
embedding_similarity(model, tokenizer, sentences)

1,2번 문장이 73.72628021240234로 가장 유사합니다.
( 73.72628021240234 18.873348236083984 17.086091995239258 )


## txt 파일로 저장되어있는 query set에 대한 유사도 테스트

In [35]:
from ast import literal_eval

with open('query.txt', 'r') as file:
    
    file_contents = file.read()
    query_list = literal_eval(file_contents)

query_list


[['치타가 들판을 가로 질러 먹이를 쫓는다.', '치타 한 마리가 먹이 뒤에서 달리고 있다.', '원숭이 한 마리가 드럼을 연주한다.'],
 ['이번 주 일요일에 분당 이마트 점은 문을 여나요?',
  '일요일에 분당 이마트는 문 열어요?',
  '분당 이마트 점은 토요일에 몇 시까지 하나요?'],
 ['빠른 갈색 여우가 게으른 개를 뛰어넘는다', '날쌘 갈색 여우가 졸린 개를 넘어간다', '태양은 동쪽에서 뜨고 서쪽에서 진다'],
 ['인공지능은 세계를 변화시키고 있다', 'AI는 우리 행성을 혁신하고 있다', '사과는 맛있고 영양가 있는 과일이다'],
 ['파이썬은 인기 있는 프로그래밍 언어이다',
  '많은 개발자들이 소프트웨어 개발을 위해 파이썬을 사용한다',
  '코끼리는 가장 큰 육상 동물이다'],
 ['커피는 인기 있는 음료이다', '많은 사람들이 아침에 커피를 마신다', '산에는 여러 종류의 나무가 있다'],
 ['데이터 과학은 정보를 분석하는 분야이다',
  '많은 회사들이 데이터 분석을 위해 데이터 과학자를 고용한다',
  '바다는 광대하고 신비로운 곳이다'],
 ['서울은 대한민국의 수도이다', '서울에는 다양한 문화와 역사가 존재한다', '사막은 건조하고 모래가 많은 지역이다'],
 ['성남에는 편의점이 몇 개가 있을까?', '너가 자주 가는 편의점은 성남에 있니?', '서울은 편의시설이 얼마나 많은가?'],
 ['컴퓨터 프로그래밍은 창의적인 과정이다',
  '많은 사람들이 소프트웨어 개발을 배운다',
  '산책하며 신선한 공기를 마시는 것은 건강에 좋다'],
 ['수학은 문제 해결에 중요하다', '공학 분야에서 수학은 필수적이다', '영화 감상은 좋은 여가 활동이다'],
 ['건강한 식단은 삶의 질을 향상시킨다', '많은 사람들이 영양가 있는 음식을 찾는다', '별을 관찰하는 것은 매력적인 취미이다'],
 ['운동은 몸과 마음에 좋다', '규칙적인 운동은 건강을 유지하는데 도움이 된다', '좋은 책을 읽는 것은 지식을 넓히는데 도움이 

In [45]:
for i, query in enumerate(query_list):
    print(f'{i})\n{query}')
    embedding_similarity(model, tokenizer, query)
    print() 
    

0)
['치타가 들판을 가로 질러 먹이를 쫓는다.', '치타 한 마리가 먹이 뒤에서 달리고 있다.', '원숭이 한 마리가 드럼을 연주한다.']
1,2번 문장이 73.72628021240234로 가장 유사합니다.
( 73.72628021240234 18.873348236083984 17.086091995239258 )

1)
['이번 주 일요일에 분당 이마트 점은 문을 여나요?', '일요일에 분당 이마트는 문 열어요?', '분당 이마트 점은 토요일에 몇 시까지 하나요?']
1,2번 문장이 94.49633026123047로 가장 유사합니다.
( 94.49633026123047 79.33795166015625 80.82059478759766 )

2)
['빠른 갈색 여우가 게으른 개를 뛰어넘는다', '날쌘 갈색 여우가 졸린 개를 넘어간다', '태양은 동쪽에서 뜨고 서쪽에서 진다']
1,2번 문장이 72.79332733154297로 가장 유사합니다.
( 72.79332733154297 13.187127113342285 4.86197566986084 )

3)
['인공지능은 세계를 변화시키고 있다', 'AI는 우리 행성을 혁신하고 있다', '사과는 맛있고 영양가 있는 과일이다']
1,2번 문장이 81.8354263305664로 가장 유사합니다.
( 81.8354263305664 -4.600234508514404 0.3070428967475891 )

4)
['파이썬은 인기 있는 프로그래밍 언어이다', '많은 개발자들이 소프트웨어 개발을 위해 파이썬을 사용한다', '코끼리는 가장 큰 육상 동물이다']
1,2번 문장이 74.05497741699219로 가장 유사합니다.
( 74.05497741699219 3.623605966567993 -7.154612064361572 )

5)
['커피는 인기 있는 음료이다', '많은 사람들이 아침에 커피를 마신다', '산에는 여러 종류의 나무가 있다']
1,2번 문장이 63.728668212890625로 가장 유사합니다.
( 63.72